In [13]:
import math
import nbimporter
import CoolProp.CoolProp as CP

### Problem 11.5. Fume hood pressure drop
 Two identical exhaust hoods are installed in a manufacturing plant to remove the gases from a welding process. Each hood is required to remove 500 cfm from the work area. A filter is placed in the duct to prevent solid particles from damaging the fan. A pressure drop measurement across the filter is used to determine when to replace the filter. The loss coefficient for a new filter is 5.5 and the loss coefficient for each fume hood is 3.3, and the fan is sized so that the system flow for a new filter is 1200 cfm.\
a. Determine the pressure loss (inches water) between the inlet to the fume hoods and the inlet to the fan for the design conditions.\
b. Determine the value of the pressure drop that would necessitate a change of filter.\
c. One of the workstations is no longer needed so a damper located in the hood is closed. Determine the system pressure loss curve for this situation. Show how the operating point changes using a representative fan characteristic curve based on both workstations in operation.

1) Information given:

In [14]:
P_atm = 14.7 # atmosphere pressure
T_z = 75 
D_1 = 9/12 # convert diameter from in to feet
V_dot_1 = 500 # cfm
L_1 = 15 # ft
D_2 = 12/12 # convert diameter from in to feet
L_2 = 10 #ft
K_hood = 3.3 #loss coefficient for each fume hood

2. Calculate the air viscosity and density at standard air pressure and 75F drybulb temperature

In [15]:
mu = CP.PropsSI("V", "P", 101325, "T", 297, "Air")* 2419.0883 # viscosity, lbm/hr-ft
rho = CP.PropsSI("D", "P", 101325, "T", 297, "Air")*0.06243 # density of air. lbm/ft^3 
print ("viscosity =", mu,'[lbm/hr-ft]')
print ("density =", rho, '[lbm/ft^3]')

viscosity = 0.0444930334982871 [lbm/hr-ft]
density = 0.07422407519055603 [lbm/ft^3]


In [16]:
Ac_1 = math.pi * (D_1)**2/4
v_1 = Ac_1 * V_dot_1 * 60 # lbm/hr
m_1 = V_dot_1